In [92]:
# import classes from folder app/classes.py
from app.libs.classes import Shift, Nurse
import json


In [93]:
with open('data/shifts.json', 'r') as f:
    shifts = json.load(f)

shifts = {shift['shift_id']: Shift(**shift) for shift in shifts}


In [94]:
with open('data/nurses.json', 'r') as f:
    nurses = json.load(f)

nurses = {nurse['nurse_id']: Nurse(**nurse) for nurse in nurses}
nurses

{1: Nurse(nurse_id=1, nurse_name='Otalora_Castro_Rocio_del_Pilar', shift_preference='morning', accumulated_hours=-9, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[4, 12, 18, 19], vacations=[]),
 2: Nurse(nurse_id=2, nurse_name='Lina_Mabel_Torres_Pulido', shift_preference='morning', accumulated_hours=-26.5, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[5, 11, 25, 26], vacations=[]),
 3: Nurse(nurse_id=3, nurse_name='Rodriguez_Bohorquez_Luz_Dary', shift_preference='afternoon', accumulated_hours=-7, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[4, 12, 18, 19, 26], vacations=[]),
 4: Nurse(nurse_id=4, nurse_name='Carrero_Contreras_Tania_Maryely', shift_preference='morning', accumulated_hours=

# Additional parameters 

In [95]:
DM = 8
HMM = 216

# Modelo

In [96]:
from datetime import datetime, timedelta
from math import floor
from typing import Dict

from app.libs.classes import Shift, Nurse
from app.libs.constants import HMM, DM
import pulp as plp
from itertools import product

In [97]:
import pulp as plp

model = plp.LpProblem("Nurse Scheduling", plp.LpMinimize)
I = set(nurses.keys())
J = set(shifts.keys())
K = set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J])

valid_keys = [(i, j) for (i,j) in product(I, J) if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day not in nurses[i].vacations]


/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [98]:
X = plp.LpVariable.dicts("X", valid_keys, cat=plp.LpBinary)
W = plp.LpVariable.dicts("W", [(i, k) for (i,k) in product(I, K)], cat=plp.LpBinary)

V = plp.LpVariable.dicts("V", I, cat=plp.LpContinuous)  # overtime
Y = plp.LpVariable.dicts("Y", I, cat=plp.LpContinuous) # TOTAL NUMBER OF SHIFTS ASSIGNED TO NURSE i
MDH = plp.LpVariable("MDH", cat=plp.LpContinuous) # MAXIMUM DIFFERENCE BETWEEN MAXIMUM SHIFTS AND REAL NUMBER OF SHIFTS ASSIGNED
DOff = plp.LpVariable.dicts("DOff", I, cat=plp.LpContinuous) 


# Objective function

In [99]:
expr_PDL = plp.lpSum([(X[i, j] for (i,j) in valid_keys if shifts[j].shift_date in nurses[i].dates_off)])
# accounts for the number of weekends a person is assigned to a shif not in his/her preference
expr_PWE = plp.lpSum([(X[i, j]) for (i,j) in valid_keys if
                        datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6] and
                        (nurses[i].shift_preference != shifts[j].shift_type)])

Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)

In [100]:
# accounts the number of shifts assigned to a nurse
for nurse in I:
    model+=Y[nurse] == plp.lpSum([X[nurse, shift] for shift in J if (nurse, shift) in valid_keys]), f"assigned_shifts_{nurse}"

In [101]:
 # accounts for the difference between shifts that should be assigned to a caregiver 
            # (considering overtime hours to be balanced) and real number of shifts assigned

for nurse in I:
    model += Y[nurse] - (HMM - nurses[nurse].accumulated_hours) * (1 / DM) <= MDH, f"balance_hours_1_{nurses[nurse].nurse_id}"
    model += (HMM - nurses[nurse].accumulated_hours) * (1 / DM) - Y[nurse] <= MDH, f"balance_hours_2_{nurses[nurse].nurse_id}"

In [102]:
# dictionary enumerating the shifts happening in each day
shifts_per_day = {}
for shift in J:
    if shifts[shift].shift_date not in shifts_per_day:
        shifts_per_day[shifts[shift].shift_date] = []
    shifts_per_day[shifts[shift].shift_date].append(shift)


In [103]:
for i in I:
    for spd in shifts_per_day.keys():
        model += plp.lpSum([X[i, j] for j in shifts_per_day[spd] if (i, j) in valid_keys]) <= 1, f"one_shift_per_day_{nurses[i].nurse_id}_{spd}"


In [104]:
# accounts for overtime V hours required
for i in I:
    model += Y[i] * DM <= (HMM - nurses[i].accumulated_hours) + V[i], f"overtime_{nurses[i].nurse_id}"

In [105]:
# demand is satisfied
for j in J:
    model += plp.lpSum([X[(i, j)] for i in I if (i, j) in valid_keys]) >= shifts[j].demand, f"demand_{j}"
        

In [106]:
# for each weekend, a nurse works at most one day
for i in I:
    for j1 in J:
        if (i, j1) in valid_keys and datetime.strptime(shifts[j1].shift_date,'%Y-%m-%d').weekday() == 5:
            model += X[i, j1] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d').weekday() == 6
                                                        and datetime.strptime(shifts[j2].shift_date,'%Y-%m-%d') == datetime.strptime(shifts[j1].shift_date, '%Y-%m-%d') + timedelta(days=1)]) <= 1, f"weekend_{nurses[i].nurse_id}_{j1}"


In [107]:
for i,k in product(I, K):
    model += (1 - plp.lpSum([X[i, j] for j in J if (i, j) in valid_keys and datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day == k])) ==W[i, k], f"day_is_off_{nurses[i].nurse_id}_{k}"

In [108]:
days = list([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6]])

total_weekend_days = len(days)

for i in I:
    model += DOff[i] == plp.lpSum([W[i, k] for k in days]), f"weekend_off_1_{nurses[i].nurse_id}"
    model += DOff[i] >= floor(total_weekend_days / 2) + 1, f"weekend_off_2_{nurses[i].nurse_id}"

In [109]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/c29e47afd5f644f99a6f69e0207e98c1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/c29e47afd5f644f99a6f69e0207e98c1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 927 COLUMNS
At line 12089 RHS
At line 13012 BOUNDS
At line 14775 ENDATA
Problem MODEL has 922 rows, 1762 columns and 7688 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 233 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 250 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 250 strengthened rows, 0 substitutions
Cgl0003I 0 fixed

1